In [1]:
import pandas as pd
import numpy as np
import openrouteservice as ors
from dotenv import load_dotenv
from pyonemap import OneMap
import os
import requests
import json
import time
from collections import namedtuple
import connectivity_functions as cf
from datetime import datetime,timedelta

In [3]:
load_dotenv()

True

In [4]:
directory = os.getcwd()

os.chdir(directory)

priv_centroid_df = pd.read_csv(r"..\data\priv_cluster_centroids.csv",header = None,names = ['Latitude','Longitude'])
mrt_stations_df = pd.read_csv(r"..\data\mrt_station_final.csv",usecols = [1,2,3])

In [4]:
priv_centroid_df.info()
mrt_stations_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550 entries, 0 to 549
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Latitude   550 non-null    float64
 1   Longitude  550 non-null    float64
dtypes: float64(2)
memory usage: 8.7 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   MRT.Name   175 non-null    object 
 1   Latitude   175 non-null    float64
 2   Longitude  175 non-null    float64
dtypes: float64(2), object(1)
memory usage: 4.2+ KB


In [5]:
mrt_stations_df.sort_values(by='MRT.Name', inplace=True)

mrt_stations_df

,MRT.Name,Latitude,Longitude
12,ADMIRALTY MRT STATION,1.440589,103.800990
9,ALJUNIED MRT STATION,1.316433,103.882906
52,ANG MO KIO MRT STATION,1.369429,103.849455
115,BAKAU LRT STATION,1.387994,103.905415
54,BANGKIT LRT STATION,1.380022,103.772647
...,...,...,...
158,WOODLANDS SOUTH MRT STATION,1.427488,103.792730
80,WOODLEIGH MRT STATION,1.339190,103.870818
77,YEW TEE MRT STATION,1.397298,103.747358
79,YIO CHU KANG MRT STATION,1.381499,103.845171


In [6]:
#create a psuedo index for my residential centroids df
priv_centroid_df['index'] = priv_centroid_df.index

priv_centroid_df['index']

#create a psuedo index for my residential centroids df
mrt_stations_df['index'] = mrt_stations_df.index

mrt_stations_df['index']

##create a dummy variable to cross join on 
priv_centroid_df['join_key'] = "A"
mrt_stations_df['join_key'] = "A"

#Cross join to obtain combinations of all possible pairings between MRTs and Residential Centroids
combined_df = pd.merge(priv_centroid_df, mrt_stations_df, on='join_key')

print(combined_df)

       Latitude_x  Longitude_x  index_x join_key                     MRT.Name  \
0        1.416601   103.759368        0        A        ADMIRALTY MRT STATION   
1        1.416601   103.759368        0        A         ALJUNIED MRT STATION   
2        1.416601   103.759368        0        A       ANG MO KIO MRT STATION   
3        1.416601   103.759368        0        A            BAKAU LRT STATION   
4        1.416601   103.759368        0        A          BANGKIT LRT STATION   
...           ...          ...      ...      ...                          ...   
96245    1.318428   103.907190      549        A  WOODLANDS SOUTH MRT STATION   
96246    1.318428   103.907190      549        A        WOODLEIGH MRT STATION   
96247    1.318428   103.907190      549        A          YEW TEE MRT STATION   
96248    1.318428   103.907190      549        A     YIO CHU KANG MRT STATION   
96249    1.318428   103.907190      549        A           YISHUN MRT STATION   

       Latitude_y  Longitud

In [7]:
#Defining a function that calculates the Euclidean Distance between two points using Haversine Method?
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0  # Earth radius in kilometers

    lat1 = np.radians(lat1)
    lon1 = np.radians(lon1)
    lat2 = np.radians(lat2)
    lon2 = np.radians(lon2)

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arcsin(np.sqrt(a))

    distance = R * c
    return distance

In [8]:
coordinate_pair = namedtuple('coordinate_pair',['lat_x','lon_x','lat_y','lon_y'])

In [9]:
#Apply function to dataframe and store distances in new column 'euclidean distance'
combined_df['euclidean_distance'] = haversine(combined_df['Latitude_x'], combined_df['Longitude_x'], combined_df['Latitude_y'], combined_df['Longitude_y'])

#Group by residential centroid, filter out the closest MRT by distance for each centroid into another dataframe and reset its index
result_df = combined_df.loc[combined_df.groupby('index_x')['euclidean_distance'].idxmin()].reset_index()

result_df

#Create a new column 'coordinate_pair' to store coordinate pairs to pass to openrouteservice API direction query
# result_df['coordinate_pair'] = list(zip(result_df['Longitude_x'], result_df['Latitude_x'], result_df['Longitude_y'], result_df['Latitude_y']))
# result_df['coordinate_pair'] = result_df['coordinate_pair'].apply(lambda x: [[x[0], x[1]], [x[2], x[3]]])
result_df['coordinate_pair'] = result_df.apply(lambda x: coordinate_pair(x['Latitude_x'], x['Longitude_x'], x['Latitude_y'], x['Longitude_y']), axis=1)

#create an empty column 'route' to later store query response
result_df['cycle_route'] = np.nan
result_df['bus_route'] = np.nan

In [5]:
one_map_email = os.getenv("ONE_MAP_EMAIL")
one_map_password = os.getenv("ONE_MAP_PASSWORD")
payload = {
        "email": one_map_email,
        "password": one_map_password
      }
api_key = requests.request("POST", "https://www.onemap.gov.sg/api/auth/post/getToken", json=payload)
api_key = api_key.json()["access_token"]

In [6]:
onemap = OneMap(api_key)

In [12]:
def get_cycle_route(coordinate_pair):
    time.sleep(0.5)
    try:
        return onemap.routing.route(start_lat=coordinate_pair.lat_x, 
                                    start_lon=coordinate_pair.lon_x, 
                                    end_lat=coordinate_pair.lat_y, 
                                    end_lon=coordinate_pair.lon_y, 
                                    routeType="cycle")
    except Exception as e:
        print(f"Error: {e}")
        return None
    
    # def get_route(coordinate_pair):
#     time.sleep(2)
#     try:
#         return client.directions(coordinate_pair, profile='cycling-regular', format='geojson', validate=False)
#     except Exception as e:
#         print(f"Error: {e}")
#         return None

In [13]:
def get_bus_route(coordinate_pair):
    # Get today's date
    today = datetime.now()
    # Calculate yesterday's date
    yesterday = today - timedelta(days=1)
    time.sleep(0.5)
    try:
        return onemap.routing.route(start_lat=coordinate_pair.lat_x, 
                                    start_lon=coordinate_pair.lon_x, 
                                    end_lat=coordinate_pair.lat_y, 
                                    end_lon=coordinate_pair.lon_y, 
                                    routeType="pt",
                                    mode = "BUS",
                                    date =  yesterday.strftime('%m-%d-%Y'),
                                    time = '18:00:00',
                                    maxWalkDistance = 2000)
    except Exception as e:
        print(f"Error: {e}")
        return None

In [14]:
result_df['cycle_route'] = result_df['coordinate_pair'].apply(get_cycle_route)

Error: 502 Server Error: Bad Gateway for url: https://www.onemap.gov.sg/api/public/routingsvc/route?start=1.385236095590498%2C103.87367777522516&end=1.391885892%2C103.8763086&routeType=cycle


In [15]:
#Not working
#result_df['bus_route'] = result_df['coordinate_pair'].apply(get_bus_route)

In [16]:
def get_distance(route):
    try:
        return route['route_summary']['total_distance']/1000 #convert m to km
    except (KeyError, IndexError,TypeError) as e:
        print(f"Error: {e}")
        return None

result_df['distance'] = result_df['cycle_route'].apply(get_distance)

def get_time(route):
    try:
        return route['route_summary']['total_time']/60 #convert second to minutes
    except (KeyError, IndexError,TypeError) as e:
        print(f"Error: {e}")
        return None
    

result_df['cycle_duration'] = result_df['cycle_route'].apply(get_time)


def get_bus_time(route):
    try: 
        return route['plan']['itineraries'][0]['duration']/60 # convert seconds to minutes
    except (KeyError, IndexError,TypeError) as e:
        print(f"Error: {e}")
        return None

result_df['bus_duration'] = result_df['cycle_route'].apply(get_bus_time)

# def get_distance(route):
#     try:
#         return route['features'][0]['properties']['segments'][0]['distance']
#     except (KeyError, IndexError,TypeError) as e:
#         print(f"Error: {e}")
#         return None

# result_df['distance'] = result_df['route'].apply(get_distance)

# def get_time(route):
#     try:
#         return route['features'][0]['properties']['segments'][0]['duration']
#     except (KeyError, IndexError,TypeError) as e:
#         print(f"Error: {e}")
#         return None
    

# result_df['duration'] = result_df['route'].apply(get_time)

Error: 'NoneType' object is not subscriptable
Error: 'NoneType' object is not subscriptable
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan

In [17]:
result_df['time_difference'] =  result_df['cycle_duration'] - result_df['bus_duration']

In [18]:
result_df

,index,Latitude_x,Longitude_x,index_x,join_key,MRT.Name,Latitude_y,Longitude_y,index_y,euclidean_distance,coordinate_pair,cycle_route,bus_route,distance,cycle_duration,bus_duration,time_difference
0,81,1.416601,103.759368,0,A,KRANJI MRT STATION,1.425087,103.762137,51,0.992547,"(1.416601020587262, 103.75936800711666, 1.4250...",{'status_message': 'Found route between points...,NaN,2.957,18.483333,None,NaN
1,251,1.323264,103.914492,1,A,KEMBANGAN MRT STATION,1.321038,103.912948,13,0.301187,"(1.3232635413132667, 103.9144924273122, 1.3210...",{'status_message': 'Found route between points...,NaN,0.463,3.300000,None,NaN
2,415,1.321304,103.792035,2,A,HOLLAND VILLAGE MRT STATION,1.312240,103.796399,56,1.118509,"(1.3213040548093986, 103.79203541849168, 1.312...",{'status_message': 'Found route between points...,NaN,1.906,12.133333,None,NaN
3,614,1.347860,103.864701,3,A,LORONG CHUAN MRT STATION,1.351639,103.863946,135,0.428542,"(1.3478599940641125, 103.86470096502855, 1.351...",{'status_message': 'Found route between points...,NaN,1.236,8.066667,None,NaN
4,787,1.380629,103.832771,4,A,LENTOR MRT STATION,1.385507,103.835744,153,0.635097,"(1.3806291442438563, 103.83277097403868, 1.385...",{'status_message': 'Found route between points...,NaN,0.873,5.266667,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545,95521,1.316019,103.830424,545,A,STEVENS MRT STATION,1.320066,103.826024,24,0.664516,"(1.3160194176441267, 103.8304236231462, 1.3200...",{'status_message': 'Found route between points...,NaN,0.911,5.766667,None,NaN
546,95690,1.334950,103.800228,546,A,SIXTH AVENUE MRT STATION,1.330858,103.796907,26,0.585992,"(1.3349498403627411, 103.80022829300584, 1.330...",{'status_message': 'Found route between points...,NaN,0.839,6.066667,None,NaN
547,95780,1.387122,103.866946,547,A,FERNVALE LRT STATION,1.391886,103.876309,112,1.167796,"(1.3871223906836447, 103.8669460348608, 1.3918...",{'status_message': 'Found route between points...,NaN,2.114,13.533333,None,NaN
548,95976,1.326026,103.912469,548,A,KEMBANGAN MRT STATION,1.321038,103.912948,13,0.557140,"(1.3260256805286963, 103.9124689604731, 1.3210...",{'status_message': 'Found route between points...,NaN,1.060,6.900000,None,NaN


In [19]:
def get_centroid_name(row):
    geocode = onemap.reverseGeocode.revGeoCode(row['Latitude_x'], row['Longitude_x'])
    if geocode['GeocodeInfo'][0]['BUILDINGNAME'] != "NIL" and geocode['GeocodeInfo'][0]['BUILDINGNAME'] != "MULTI STOREY CAR PARK":
            return geocode['GeocodeInfo'][0]['BUILDINGNAME']
    else:
        if geocode['GeocodeInfo'][0]['BLOCK'] != "NIL":
            return geocode['GeocodeInfo'][0]['BLOCK'] + " " + geocode['GeocodeInfo'][0]['ROAD']
        else:
            return geocode['GeocodeInfo'][0]['ROAD']

In [20]:
result_df['centroid_name'] = result_df.apply(get_centroid_name, axis=1)

In [7]:
ors_key = os.getenv("ORS_API_KEY")

client = ors.Client(key= ors_key)

In [24]:
result_df['suitability'] = result_df['cycle_route'].apply(lambda x: cf.get_path_suitability(client,x))


TypeError: 'NoneType' object is not subscriptable

In [ ]:
result_df['steepness'] = result_df['cycle_route'].apply(lambda x: cf.get_path_steepness(client,x))

In [ ]:
new_result_df = result_df. loc[:, result_df. columns != 'route']
#new_result_df.to_csv(r"..\data\Private_Centroid_MRT pairing data.csv")
new_result_df.to_csv(r"C:\Users\leoqi\Downloads\Private_Centroid_MRT pairing data.csv")
routes = result_df['cycle_route'].copy(deep = True)
#routes.to_json(r'..\data\Private_MRT_cycle_routes.json', orient='records')
routes.to_json(r'C:\Users\leoqi\Downloads\Private_MRT_cycle_routes.json', orient='records')

In [ ]:
result_df[result_df['route'].isna()]

,index,Latitude_x,Longitude_x,index_x,join_key,MRT.Name,Latitude_y,Longitude_y,index_y,euclidean_distance,coordinate_pair,route,distance,duration,centroid_name


In [ ]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275 entries, 0 to 274
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   index               275 non-null    int64  
 1   Latitude_x          275 non-null    float64
 2   Longitude_x         275 non-null    float64
 3   index_x             275 non-null    int64  
 4   join_key            275 non-null    object 
 5   MRT.Name            275 non-null    object 
 6   Latitude_y          275 non-null    float64
 7   Longitude_y         275 non-null    float64
 8   index_y             275 non-null    int64  
 9   euclidean_distance  275 non-null    float64
 10  coordinate_pair     275 non-null    object 
 11  route               275 non-null    object 
 12  distance            275 non-null    float64
 13  duration            275 non-null    float64
 14  centroid_name       275 non-null    object 
dtypes: float64(7), int64(3), object(5)
memory usage: 32.4+ KB